# Coupling BEM-FEM-BEM with Cavity.

<img src="2_Interfaces_Cav_EG.png" width="500" height="250">

To calculate the solvation energy of a solute using the BEM-FEM-BEM with cavity, which is equivalent to the BEM-BEM-FEM-BEM coupling, the following matrix system must be solved, which comes from solving the following PDE, including its border conditions:
\begin{equation} 
\left\{\begin{matrix} 
     -\nabla^{2}\phi_{s_{0}}(x)+\kappa_{s}^{2}\phi_{s_{0}}(x) =0 \ & x \ \in \ \Omega_{s_{0}} \\ 
     -\epsilon_{m}\nabla^{2} \phi_{m}(x)=\sum_{i=1}^{n_{c}}Q_{i}\delta\left (x-x_{i} \right ) & x \ \in \ \Omega_{m} \\
     -\nabla\cdot(\epsilon_{i}(x)\nabla\phi_{i}(x))+\bar{\kappa_{i}}^{2}(x)\phi_{i}(x)
     =0 \ & x \ \in \ \Omega_{i} \\
     -\nabla^{2}\phi_{s}(x)+\kappa_{s}^{2}\phi_{s}(x)=0 & x \ \in \ \Omega_{s} \\
     \phi_{s_{0}}(x)= \phi_{m}(x) & x \ \in \ \Gamma_{c} \\
     \epsilon_{s}\partial_n^c \phi_{s_{0}}(x)= \epsilon_{m}\partial_n^c \phi_{m}(x) & x \ \in \ \Gamma_{c} \\
     \phi_{m}(x)= \phi_{i}(x) & x \ \in \ \Gamma_{a} \\
     \epsilon_{m}\partial_n^a \phi_{m}(x)= \epsilon_{i}(x)\partial_n^a \phi_{i}(x) & x \ \in \ \Gamma_{a} \\
     \phi_{i}(x)= \phi_{s}(x) & x \ \in \ \Gamma_{b} \\
     \epsilon_{i}(x)\partial_n^b \phi_{i}(x)= \epsilon_{s}\partial_n^b \phi_{s}(x) & x \ \in \ \Gamma_{b}
\end{matrix}\right.
\end{equation}

Where $\Omega_{m}$ is the solute domain, $\Omega_{i}$ is the Stern Layer, $\Omega_{s}$ is the solvent domain, $\Omega_{s_0}$ is the cavity domain, $\Gamma_{a}$ is the inner interface, $\Gamma_{b}$ is the outer interface and $\Gamma_{c}$ is the cavity-solute interface. And its matrix system corresponds to:
\begin{equation}
\begin{bmatrix}
\frac{\epsilon_{m}}{\epsilon_{s}}V_{H,c}^{c} & \frac{1}{2}I-K_{H,c}^{c} & 0 & 0 & 0\\ 
-V_{L,c}^{c} & \frac{1}{2}I+K_{L,c}^{c} & V_{L,a}^{c} & -K_{L,a}^{c} & 0\\ 
-V_{L,c}^{a} & K_{L,c}^{a} & V_{L,a}^{a} & \frac{1}{2}I-K_{L,a}^{a} & 0\\ 
0 & 0 &-\epsilon_{m}M_{\Gamma_{a}} & \epsilon_{i}(x)A_{\Omega_{i}}+\bar{\kappa_{i}}^{2}(x)M_{\Omega_{i}} & -\epsilon_{s}M_{\Gamma_{b}}\\ 
0 & 0 & 0 & \frac{1}{2}I-K_{H}^{b} & V_{H}^{b}
\end{bmatrix} \cdot
\begin{bmatrix}
\partial_n \phi_{m}^{\Gamma_{c}}\\ 
\phi_{m}^{\Gamma_{c}} \\
\partial_n \phi_{m}^{\Gamma_{a}}\\ 
\phi_{i}^{\Omega_{i}} \\
\partial_n \phi_{s}^{\Gamma_{b}} \\ 
\end{bmatrix} =
\begin{bmatrix}
0 \\
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{c}}-x_{i}\right \|} \\ 
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{a}}-x_{i}\right \|} \\ 
0\\ 
0 \\
\end{bmatrix}
\end{equation}

Where $\epsilon_i(x)$ and $\bar{\kappa_{i}}^{2}(x)$, corresponds to the behavior of the permittivity and the modified Debye-Huckel parameter in the FEM domain in $\Omega_{i}$, where for this coupling, we have the constant function, the linear function and the hyperbolic tangent.
\begin{equation} 
\textbf{Constant}\left\{ 
\begin{matrix}
\epsilon_i(x)=\epsilon_i  \\ 
\bar{\kappa_{i}}^{2}(x) =\epsilon_{i}\kappa_{i}^{2} \\
\end{matrix} \right. \quad
\textbf{Linear}\left\{ 
\begin{matrix}
\epsilon_i(x)=\epsilon_m + (\epsilon_s -\epsilon_m )\alpha(x)  \\ 
\bar{\kappa_{i}}^{2}(x) =\epsilon_{s}\kappa_{s}^{2}\alpha(x) \\
\end{matrix} \right. \quad
\textbf{Hyperbolic Tangent}\left\{ 
\begin{matrix}
S(x,k_{p}) = \frac{1}{2}-\frac{1}{2}tanh\left (k_{p}\left (\alpha(x)-\frac{1}{2} \right )\right ) \\ 
\epsilon_i(x)=S(x,k_{p})\epsilon_m +(1-S(x ,k_{p}))\epsilon_s \\
\bar{\kappa_{i}}^{2}(x) = (1-S(x ,k_{p}))\epsilon_{s}\kappa_{s}^{2} 
\end{matrix} \right.
\end{equation}

And when solving the matrix system, the solvation energy is calculated as:
\begin{equation} 
    \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i} \left ( V_L\partial_n \phi_m^{\Gamma_{c}}(x_{i})-K_L\phi_m^{\Gamma_{c}}(x_{i})+K_L\phi_i^{\Gamma_{a}}(x_{i}) - V_L\partial_n \phi_m^{\Gamma_{a}}(x_{i})\right )
\end{equation}   

## Implementation

First we import the main libraries such as Numpy, Trimesh, Bempp and Dolfin, as well as files in Python to read the solute files, which are in ".pqr" and ".off" format.

In [1]:
import bempp.api
import numpy as np 
import dolfin
from readoff import *
from readpqr import *
import trimesh
import time
start = time.time()

Next, the physical conditions of the problem in each region are defined, such as pemitivity and
Debye-Huckel parameter.

In [2]:
em = 4.           #[-] Interior electrical permittivity of the solute.
es = 80.          #[-] Exterior electrical permittivity of the solvent.
ks = 0.125        #[1/A] Inverse of the Debye-Huckel length of the fluid in the solvent.

We choice of variable permittivity method in the intermediate domain, along with other parameters necessary for each function.

In [3]:
# Variable function.
# 'C' if it is Constant.
# 'VL' if it is for Linear variable.
# 'VTH' if it is by Hyperbolic Tangent variable.
Va = 'VTH' 

#For the case of Hyperbolic Tangent.
kp = 'N'         #[-] Transition slope of the Hyperbolic Tangent from 0 to infinity. 'N'for kp tending to infinity.
#For the constant.
ei = (es+em)/2              #[-] Intermediate electrical permittivity of the intermediate domain.
ki = ks*np.sqrt(es/(es+em)) #[1/A] Inverse of the Debye-Huckel length of the fluid in the intermediate domain.

Later, we choose the solution that we want to use to calculate the energy, where we import the volumetric mesh, the ".pqr" file to obtain the information on the radii, charges and position of the solute atoms, and a text file to with the alpha values to work with the variable function, if it necessary.

In [4]:
MeshV = 'Sphere/Mallas_V/outputTetMeshSphere58R4T11.xml'  #Path of the volumetric mesh of the solute for which the energy is to be calculated. In "xml" format.
PQR  =  'Sphere/PQR/Sphere5Q3.pqr'              #Path of the position and charges of the solute for which the energy is to be calculated. In "pqr" format.
FileA = 'Sphere/Lista_A/Alfa_Sphere58R4T11.txt' #Path of the solute alpha values file to work with variable permittivities. In "txt" format. If not necessary, place ''.
Mesh_C = 'Sphere/Mallas_S/SphereCavityR4.off' #Path of the original mesh with cavities or a mesh with only cavities.
PC,Q,R = readpqr(PQR) 

In addition, we can choose the type of assembly of the edge operators, parameters of the GMRES and the FMM if required.

In [5]:
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =100     #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4  

We generate a volumetric mesh with Dolfin. Later, the boundary mesh will be extracted from this.

In [6]:
from dolfin import Mesh
mesh = Mesh(MeshV)  #Creation of the volumetric mesh.

Next, we generate the surface mesh of the cavity.

In [7]:
vertices_C,faces_C  = read_off(Mesh_C) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = vertices_C, faces= faces_C) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split)) 

if len(mesh_split)>=2:
    #Sort surface meshes of the cavities by number of vertices.
    LMS = []
    for i in range(len(mesh_split)):
        LMS.append(len(mesh_split[i].vertices))
    IMS = np.argsort(LMS)[::-1]  
    #Unite the information from the individual meshes of the cavities into a single surface mesh. 
    MS = mesh_split[IMS[1]]
    if len(mesh_split)>2: 
        for i in range(len(mesh_split)-2):
            MS = MS + mesh_split[IMS[i+2]]
    vertices_3 = MS.vertices 
    faces_3 = MS.faces  
elif len(mesh_split)==1:   #Optional cavity mesh if in an '.off' file.
    vertices_3 = mesh_split[0].vertices 
    faces_3 = mesh_split[0].faces 
grid3 = bempp.api.grid.grid.Grid(vertices_3.transpose(), faces_3.transpose()) #Creation of the surface mesh with cavities.

#Generate functional spaces of the potential and its derivative in the cavity.
bempp_space0 = bempp.api.function_space(grid3, "P", 1)  #Electrostatic potential at the inner interface.
bempp_space3 = bempp.api.function_space(grid3, "P", 1)  #Derived from the electrostatic potential at the inner interface.

Found 1 meshes


We make the Bempp function spaces. The function $\textbf{fenics_to_bempp_trace_data}$ will extract the $\textbf{trace_space}$ from the Dolfin space and create the matrix $\textbf{trace_matrix}$, which maps between the dofs (degrees of freedom) in Dolfin and Bempp.

In [8]:
from bempp.api.external import fenics
fenics_space = dolfin.FunctionSpace(mesh, "CG", 1)  #Electrostatic potential at the interface and domain of the solute.
trace_space, trace_matrix = \
    fenics.fenics_to_bempp_trace_data(fenics_space) #Global trace space to work in BEM and FEM simultaneously.

Later, we process to separate the $\textbf{trace_space}$ for the case of the inner surface mesh and the outer surface mesh.

In [9]:
#Code to identify vertices and faces of the inner and outer mesh.
faces_0 = trace_space.grid.elements
vertices_0 = trace_space.grid.vertices
meshSP = trimesh.Trimesh(vertices = vertices_0.transpose(), faces= faces_0.transpose())
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))
vertices_Ref = len(mesh_split[0].vertices)
faces_Ref = len(mesh_split[0].faces)

#Obtaining the inner surface mesh.
faces_1 = faces_0.transpose()[:faces_Ref]
vertices_1 = vertices_0.transpose()[:vertices_Ref]
grid1 = bempp.api.grid.grid.Grid(vertices_1.transpose(), faces_1.transpose())
bempp_space1 = bempp.api.function_space(grid1, "P", 1) # Derived from the electrostatic potential at the inner interface.
trace_space1 = bempp.api.function_space(grid1, "P", 1) # Trace space to work at inner BEM and FEM simultaneously.

#Obtaining the outer surface mesh.
faces_2 = faces_0.transpose()[faces_Ref:]
vertices_2 = vertices_0.transpose()[vertices_Ref:]
grid2 = bempp.api.grid.grid.Grid(vertices_2.transpose(), (faces_2-len(vertices_1)).transpose())
bempp_space2 = bempp.api.function_space(grid2, "P", 1) # Derived from the electrostatic potential at the upper interface.
trace_space2 = bempp.api.function_space(grid2, "P", 1) # Trace space to work at outer BEM and FEM simultaneously.

#Element visualization.
print("FEM dofs: {0}".format(mesh.num_vertices()))
print("BEM1 dofs: {0}".format(bempp_space1.global_dof_count))
print("BEM2 dofs: {0}".format(bempp_space2.global_dof_count))
print("Tra1 dofs: {0}".format(trace_space1.global_dof_count))
print("Tra2 dofs: {0}".format(trace_space2.global_dof_count))
print("TraL dofs: {0}".format(trace_space.global_dof_count))
print("BEM0 dofs: {0}".format(bempp_space0.global_dof_count))
print("BEM3 dofs: {0}".format(bempp_space3.global_dof_count))

Found 2 meshes
FEM dofs: 50529
BEM1 dofs: 4591
BEM2 dofs: 11924
Tra1 dofs: 4591
Tra2 dofs: 11924
TraL dofs: 16515
BEM0 dofs: 180
BEM3 dofs: 180


And, we process to separate the $\textbf{trace_matrix}$ for the case of the inner surface mesh and the outer surface mesh.

In [10]:
Nodos = np.zeros(trace_space.global_dof_count)
Lista_Vertices = []

#Procedure to locate the vertices of the lower trace in the global trace.
for i in range(len(trace_space1.grid.vertices.T)):
    valores = np.linalg.norm(trace_space1.grid.vertices[:, i] - trace_space.grid.vertices.T,axis= 1)
    index = np.argmin(valores)
    Lista_Vertices.append(index)
    
Nodos[Lista_Vertices] = 1
trace_matrix1 = trace_matrix[Nodos.astype(bool)]
trace_matrix2 = trace_matrix[np.logical_not(Nodos)]

We create the boundary operators that we need for each domain. 

In [11]:
#Identity operators.
I3 = bempp.api.operators.boundary.sparse.identity(bempp_space0, bempp_space3, bempp_space3) # 1
I0 = bempp.api.operators.boundary.sparse.identity(bempp_space0, bempp_space0, bempp_space0) # 1
I1 = bempp.api.operators.boundary.sparse.identity(trace_space1, bempp_space1, bempp_space1) # 1
I2 = bempp.api.operators.boundary.sparse.identity(trace_space2, bempp_space2, bempp_space2) # 1

#Domain in the cavity in BEM.
if ks==0:
    V3 = bempp.api.operators.boundary.laplace.single_layer(bempp_space3, bempp_space3, bempp_space3, assembler=Assemble) #V 
    K3 = bempp.api.operators.boundary.laplace.double_layer(bempp_space0, bempp_space3, bempp_space3, assembler=Assemble) #K 
else:
    V3 = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space3, bempp_space3, bempp_space3, ks, assembler=Assemble) #V
    K3 = bempp.api.operators.boundary.modified_helmholtz.double_layer(bempp_space0, bempp_space3, bempp_space3, ks, assembler=Assemble) #K
Z3 = bempp.api.ZeroBoundaryOperator(bempp_space1, bempp_space3, bempp_space3) #0
Z31 = bempp.api.ZeroBoundaryOperator(trace_space1, bempp_space3, bempp_space3) #0
Z32 = bempp.api.ZeroBoundaryOperator(bempp_space2, bempp_space3, bempp_space3) #0

#Domain of the solute Ωm in the inner mesh of BEM.
V0 = bempp.api.operators.boundary.laplace.single_layer(bempp_space3, bempp_space0, bempp_space0, assembler=Assemble) #V 
K0 = bempp.api.operators.boundary.laplace.double_layer(bempp_space0, bempp_space0, bempp_space0, assembler=Assemble) #K 
V01 = bempp.api.operators.boundary.laplace.single_layer(bempp_space1, bempp_space0, bempp_space0, assembler=Assemble) #V 
K01 = bempp.api.operators.boundary.laplace.double_layer(trace_space1, bempp_space0, bempp_space0, assembler=Assemble) #K 
Z0 = bempp.api.ZeroBoundaryOperator(bempp_space2, bempp_space0, bempp_space0) #0

#Domain of the solute Ωm in the outer mesh of BEM.
V10 = bempp.api.operators.boundary.laplace.single_layer(bempp_space3, bempp_space1, bempp_space1, assembler=Assemble) #V 
K10 = bempp.api.operators.boundary.laplace.double_layer(bempp_space0, bempp_space1, bempp_space1, assembler=Assemble) #K 
V1 = bempp.api.operators.boundary.laplace.single_layer(bempp_space1, bempp_space1, bempp_space1, assembler=Assemble) #V
K1 = bempp.api.operators.boundary.laplace.double_layer(trace_space1, bempp_space1, bempp_space1, assembler=Assemble) #K
Z1 = bempp.api.ZeroBoundaryOperator(bempp_space2, bempp_space1, bempp_space1) #0

#Intermediate domain in FEM.
ZF1 = bempp.api.ZeroBoundaryOperator(bempp_space3, trace_space1, trace_space1) #0
ZF0 = bempp.api.ZeroBoundaryOperator(bempp_space0, trace_space1, trace_space1) #0
mass1 = bempp.api.operators.boundary.sparse.identity(bempp_space1, trace_space1, trace_space1) # 1
mass2 = bempp.api.operators.boundary.sparse.identity(bempp_space2, trace_space2, trace_space2) # 1

#Domain of the solvent Ωs in BEM.
Z22 = bempp.api.ZeroBoundaryOperator(bempp_space3, bempp_space2, bempp_space2) #0
Z21 = bempp.api.ZeroBoundaryOperator(bempp_space0, bempp_space2, bempp_space2) #0
Z2 = bempp.api.ZeroBoundaryOperator(bempp_space1, bempp_space2, bempp_space2) #0
if ks==0:
    K2 = bempp.api.operators.boundary.laplace.double_layer(trace_space2, bempp_space2, bempp_space2, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.laplace.single_layer(bempp_space2, bempp_space2, bempp_space2, assembler=Assemble) #V  
else:
    K2 = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space2, bempp_space2, bempp_space2, ks, assembler=Assemble) #K
    V2 = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space2, bempp_space2, bempp_space2, ks, assembler=Assemble) #V

Aditionaly, we create the Dolfin function spaces.

In [12]:
u = dolfin.TrialFunction(fenics_space)
v = dolfin.TestFunction(fenics_space)

Then, we create the different variable functions of the intermediate domain for the BEM-FEM-BEM coupling with cavity, such as the constant, the linear and the hyperbolic tangent.
\begin{equation} 
\textbf{Constant}\left\{ 
\begin{matrix}
\epsilon_i(x)=\epsilon_i  \\ 
\bar{\kappa_{i}}^{2}(x) =\epsilon_{i}\kappa_{i}^{2} \\
\end{matrix} \right. \quad
\textbf{Linear}\left\{ 
\begin{matrix}
\epsilon_i(x)=\epsilon_m + (\epsilon_s -\epsilon_m )\alpha(x)  \\ 
\bar{\kappa_{i}}^{2}(x) =\epsilon_{s}\kappa_{s}^{2}\alpha(x) \\
\end{matrix} \right. \quad
\textbf{Hyperbolic Tangent}\left\{ 
\begin{matrix}
S(x,k_{p}) = \frac{1}{2}-\frac{1}{2}tanh\left (k_{p}\left (\alpha(x)-\frac{1}{2} \right )\right ) \\ 
\epsilon_i(x)=S(x,k_{p})\epsilon_m +(1-S(x ,k_{p}))\epsilon_s \\
\bar{\kappa_{i}}^{2}(x) = (1-S(x ,k_{p}))\epsilon_{s}\kappa_{s}^{2} 
\end{matrix} \right.
\end{equation}

In [13]:
#Definition of the variable functions of the intermediate domain.
def Lista_Alfa(File): #Function to obtain the alpha parameter of the text and save it in a list.
    L_Alfa = []
    with open(File,"r") as f:  
        lines = f.readlines()
    for line in lines:
        line = line.split()
        L_Alfa.append(float(line[0]))
    return L_Alfa

#Case ei by variable by Hyperbolic Tangent. 
class Fun_ei_Tangente_Hiperbolica(dolfin.UserExpression):  
    def __init__(self,kp,L_Alfa,**kwargs):
        super().__init__(**kwargs)
    def eval_cell(self, v, x, ufc_cell):
        alfa = L_Alfa[ufc_cell.index]
        if kp=='N':  #Optional for the Hyperbolic Tangent function with kp equal to infinity.
            if alfa<0.5:
                v[0] = 1
            else:
                v[0] = 0        
        else:
            v[0] = 0.5-0.5*np.tanh(kp*(alfa-0.5)) #Hyperbolic tangent function for kp other than infinity.         
    def value_shape(self):
        return ()  

#Case ei for Linear variable.
class Fun_ei_Lineal(dolfin.UserExpression):  
    def __init__(self,L_Alfa,**kwargs):
        super().__init__(**kwargs)
    def eval_cell(self, v, x, ufc_cell):
        v[0] = L_Alfa[ufc_cell.index]  #The calculated alpha corresponds to the same linear interpolation between the two surface meshes.
    def value_shape(self):
        return ()  

if Va=='C':  #Constant Case.
    EI = ei
    K  = EI*ki**2
elif Va=='VL':  #Linear variable case.
    L_Alfa = Lista_Alfa(FileA) 
    S = Fun_ei_Lineal(L_Alfa,degree=0)
    EI = em+(es-em)*S 
    K  = S*(es*ks**2)
elif Va=='VTH': #Case of variable by Hyperbolic Tangent.
    L_Alfa = Lista_Alfa(FileA) 
    S = Fun_ei_Tangente_Hiperbolica(kp,L_Alfa,degree=0)
    EI = es+(em-es)*S
    K = (1-S)*(es*ks**2)

Later, we create the Coulomb potential function $\phi_{c}^{\Gamma}$ and with that, we construct the vector right head side $b$ of formulation.
\begin{equation}
\phi_{c}^{\Gamma} =\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma}-x_{i}\right \|} \quad \quad
b=\begin{bmatrix}
0 \\
\phi_{c}^{\Gamma_{c}} \\
\phi_{c}^{\Gamma_{a}} \\
0 \\
0 \\
\end{bmatrix}
\end{equation}

In [14]:
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uc0 = bempp.api.GridFunction(bempp_space0, fun=U_c)
Uc1 = bempp.api.GridFunction(bempp_space1, fun=U_c)

# Rhs in Ωs0 cavity in BEM.
rhs_bem3 = np.zeros(bempp_space3.global_dof_count) 
# Rhs in Ωm inner mesh in BEM.
rhs_bem0 = (Uc0).projections(bempp_space0)
# Rhs in Ωm outer mesh in BEM.
rhs_bem1 = (Uc1).projections(bempp_space1)
# Rhs in Ωi in FEM.
rhs_fem =  np.zeros(mesh.num_vertices()) 
# Rhs in Ωs in BEM.
rhs_bem2 = np.zeros(bempp_space2.global_dof_count) 
# The combination of rhs.
rhs = np.concatenate([rhs_bem3, rhs_bem0, rhs_bem1, rhs_fem, rhs_bem2])

And we construct the global left 5x5 matrix $A$ of the formulation.   
\begin{equation}
A=\begin{bmatrix}
\frac{\epsilon_{m}}{\epsilon_{s}}V_{H,c}^{c} & \frac{1}{2}I-K_{H,c}^{c} & 0 & 0 & 0\\ 
-V_{L,c}^{c} & \frac{1}{2}I+K_{L,c}^{c} & V_{L,a}^{c} & -K_{L,a}^{c} & 0\\ 
-V_{L,c}^{a} & K_{L,c}^{a} & V_{L,a}^{a} & \frac{1}{2}I-K_{L,a}^{a} & 0\\ 
0 & 0 &-\epsilon_{m}M_{\Gamma_{a}} & \epsilon_{i}(x)A_{\Omega_{i}}+\bar{\kappa_{i}}^{2}(x)M_{\Omega_{i}} & -\epsilon_{s}M_{\Gamma_{b}}\\ 
0 & 0 & 0 & \frac{1}{2}I-K_{H}^{b} & V_{H}^{b}
\end{bmatrix}
\end{equation}

In [15]:
from bempp.api.external.fenics import FenicsOperator
from scipy.sparse.linalg import LinearOperator
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
blocks = [[None,None,None,None,None],[None,None,None,None,None],[None,None,None,None,None],[None,None,None,None,None],[None,None,None,None,None]]

trace_op1 = LinearOperator(trace_matrix1.shape, lambda x:trace_matrix1*x)
trace_op2 = LinearOperator(trace_matrix2.shape, lambda x:trace_matrix2*x)
A = FenicsOperator((EI*dolfin.inner(dolfin.nabla_grad(u),dolfin.nabla_grad(v))+ K*u*v) * dolfin.dx)

#Position of the 5x5 matrix.
blocks[0][0] = V3.weak_form()*(em/es)                 
blocks[0][1] = (0.5*I3-K3).weak_form()                
blocks[0][2] = Z3.weak_form()                         
blocks[0][3] = Z31.weak_form()*trace_op1              
blocks[0][4] = Z32.weak_form()                        

blocks[1][0] = -V0.weak_form()                       
blocks[1][1] = (0.5*I0+K0).weak_form()               
blocks[1][2] = V01.weak_form()                      
blocks[1][3] = -K01.weak_form()*trace_op1            
blocks[1][4] = Z0.weak_form()                        

blocks[2][0] = -V10.weak_form()                     
blocks[2][1] = K10.weak_form()                      
blocks[2][2] = V1.weak_form()                       
blocks[2][3] = (0.5*I1-K1).weak_form()*trace_op1    
blocks[2][4] = Z1.weak_form()                       

blocks[3][0] = trace_matrix1.T*ZF1.weak_form().A               
blocks[3][1] = trace_matrix1.T*ZF0.weak_form().A               
blocks[3][2] = -trace_matrix1.T *em*mass1.weak_form().A        
blocks[3][3] =  A.weak_form()                                  
blocks[3][4] = -trace_matrix2.T *es*mass2.weak_form().A        

blocks[4][0] = Z22.weak_form()                         
blocks[4][1] = Z21.weak_form()                         
blocks[4][2] = Z2.weak_form()                          
blocks[4][3] = (0.5*I2-K2).weak_form()*trace_op2       
blocks[4][4] = V2.weak_form()                          
blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))  

For BEM-FEM-BEM coupling with cavity, we constructed the respective Mass Matrix preconditioner.

In [16]:
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import diags

P1 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space3, bempp_space3, bempp_space3).weak_form())

P2 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space0, bempp_space0, bempp_space0).weak_form())

P3 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space1, bempp_space1, bempp_space1).weak_form())
    
P4 = diags(1./(blocked[3,3].A).diagonal())
    
P5 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space2, bempp_space2, bempp_space2).weak_form())

def apply_prec(x):
    """Apply the block diagonal preconditioner"""
    m1 = P1.shape[0]
    m2 = P2.shape[0]
    m3 = P3.shape[0]
    m4 = P4.shape[0]
    m5 = P5.shape[0]
    n1 = P1.shape[1]
    n2 = P2.shape[1]
    n3 = P3.shape[1]
    n4 = P4.shape[1]
    n5 = P5.shape[1]
 
    res1 = P1.dot(x[:n1])
    res2 = P2.dot(x[n1: n1+n2])
    res3 = P3.dot(x[n1+n2:  n1+n2+n3])
    res4 = P4.dot(x[n1+n2+n3: n1+n2+n3+n4])
    res5 = P5.dot(x[n1+n2+n3+n4:])
    return np.concatenate([res1, res2, res3, res4, res5])

p_shape = (P1.shape[0] + P2.shape[0] + P3.shape[0]+ P4.shape[0] + P5.shape[0], P1.shape[1] + P2.shape[1] + P3.shape[1]+ P4.shape[1] + P5.shape[1])
P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Later, we solve the matrix system $Ax=b$ with GMRES together with its respective preconditioning an efficient solution. The solution is then divided into the parts associated with  $\partial_{n}\phi_{m}^{\Gamma_{c}}$, $\phi_{m}^{\Gamma_{c}}$, $\partial_{n}\phi_{m}^{\Gamma_{a}}$, $\phi_{i}^{\Omega_{i}}$ and $\partial_{n}\phi_{s}^{\Gamma_{b}}$.

In [17]:
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)

# Solution by GMRES.
from scipy.sparse.linalg import gmres
start1 = time.time()
soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res) 
end1 = time.time() 

# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))  

100 2.4061209299062574e-05
200 1.0723059118213043e-07
Number of GMRES iterations: 258
Total time in GMRES: 92.63 [s]


Next, we make Bempp and Dolfin functions from the solution of $\partial_{n}\phi_{m}^{\Gamma_{c}}$, $\phi_{m}^{\Gamma_{c}}$, $\partial_{n}\phi_{m}^{\Gamma_{a}}$ and $\phi_{i}^{\Omega_{i}}$, including the transformation from $\phi_{i}^{\Omega_{i}}$ to $\phi_{i}^{\Gamma_{a}}$.

In [18]:
soln_bem3 = soln[:bempp_space3.global_dof_count]
soln_bem0 = soln[bempp_space3.global_dof_count : bempp_space3.global_dof_count + bempp_space0.global_dof_count]
soln_bem1 = soln[bempp_space3.global_dof_count + bempp_space0.global_dof_count : bempp_space3.global_dof_count + bempp_space0.global_dof_count + bempp_space1.global_dof_count]
soln_fem  = soln[bempp_space3.global_dof_count + bempp_space0.global_dof_count + bempp_space1.global_dof_count : bempp_space3.global_dof_count + bempp_space0.global_dof_count + bempp_space1.global_dof_count + mesh.num_vertices()]
soln_bem2 = soln[bempp_space3.global_dof_count + bempp_space0.global_dof_count + bempp_space1.global_dof_count + mesh.num_vertices():]

# Calculate the solution of the real potential in the FEM domain in the intermediate region.
u = dolfin.Function(fenics_space)
u.vector()[:] = np.ascontiguousarray(np.real(soln_fem)) 

# Solution for Dirichlet data in the inner interface.
dirichlet_data1 = trace_matrix1 * soln_fem
dirichlet_fun1 = bempp.api.GridFunction(trace_space1, coefficients=dirichlet_data1)
# Solution for Neumann data in the inner interface.
neumann_fun1 = bempp.api.GridFunction(bempp_space1, coefficients=soln_bem1)

# Solution for Dirichlet data in the outer interface.
dirichlet_data2 = trace_matrix2 * soln_fem
dirichlet_fun2 = bempp.api.GridFunction(trace_space2, coefficients=dirichlet_data2)
# Solution for Neumann data in the outer interface.
neumann_fun2 = bempp.api.GridFunction(bempp_space2, coefficients=soln_bem2)

# Solution for Dirichlet data in the cavity.
dirichlet_fun0 = bempp.api.GridFunction(bempp_space0, coefficients=soln_bem0)
# Solution for Neumann data in the cavity.
neumann_fun0 = bempp.api.GridFunction(bempp_space3, coefficients=soln_bem3)

Finally we calculate the solvation energy for the BEM-FEM-BEM coupling with cavity, using the equation of the potential at the position of the solute atoms.
\begin{equation}
\phi_m(x) =V_L\partial_n \phi_m^{\Gamma_{c}}(x)-K_L\phi_m^{\Gamma_{c}}(x)+K_L\phi_i^{\Gamma_{a}}(x) - V_L\partial_n \phi_m^{\Gamma_{a}}(x)  \quad \quad
 \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i}\phi_{m}(x_i) 
\end{equation} 

In [19]:
#Result of the total solvation energy.
VF0 = bempp.api.operators.potential.laplace.single_layer(bempp_space3, np.transpose(PC)) 
KF0 = bempp.api.operators.potential.laplace.double_layer(bempp_space0, np.transpose(PC))
VF1 = bempp.api.operators.potential.laplace.single_layer(bempp_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(trace_space1, np.transpose(PC))
uF = VF0*neumann_fun0 - KF0*dirichlet_fun0 + KF1*dirichlet_fun1 - VF1*neumann_fun1 
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -92.515857 [kCal/mol]
Total time: 185.54 [s]
